In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../utils"))
from aoc_utils import load_data, check

In [ ]:
from functools import cache

In [ ]:
data = load_data(2023, 12)

In [ ]:
# data, part_1, part_2
tests = [
    (
        """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""",
        21,
        525152,
    ),
]

# Part 1

The first solution recurses around the central clue. It does not rely on caching.  
It processes part 2 with the full input in about 10 seconds.

In [ ]:
def leftmost(line, clues):
    if line[-1] != ".":
        line = line + "."
    pos = 0
    for c in clues:
        if pos + c >= len(line):
            return None
        while "." in line[pos:pos + c] or not line[pos + c] in ".?":
            pos += 1
            if pos + c >= len(line):
                return None
        pos += c + 1
    while pos < len(line) and line[pos] == ".":
        pos += 1
    return pos

def rightmost(line, clues):
    left = leftmost(line[::-1], clues[::-1])
    if left is None:
        return None
    return len(line) - left

def possible(line, clue, pos):
    if pos < 0 or pos + clue > len(line):
        return False
    if "." in line[pos:pos + clue]:
        return False
    if pos > 0 and line[pos - 1] == "#":
        return False
    if pos + clue < len(line) and line[pos + clue] == "#":
        return False
    return True

def line_arrangements(line, clues):
    if len(clues) == 0:
        if "#" in line:
            return 0
        return 1
    if sum(clues) + len(clues) - 1 > len(line):
        return 0
    if len(clues) == 1:
        clue = clues[0]
        line = "." + line + "."
        arr = 0
        for pos in range(1, len(line) - clue):
            if "#" not in line[:pos] and "#" not in line[pos + clue:] and "." not in line[pos:pos + clue]:
                arr += 1
        return arr
    # use the middle clue as a pivot
    pivot = len(clues) // 2
    left_clues = clues[:pivot]
    leftmost_pos = leftmost(line, left_clues)
    if leftmost_pos is None:
        return 0
    right_clues = clues[pivot + 1:]
    rightmost_pos = rightmost(line, right_clues)
    if rightmost_pos is None:
        return 0
    clue = clues[pivot]
    arr = 0
    for pos in range(leftmost_pos, rightmost_pos + 1):
        if possible(line, clue, pos):
            left_arrangements = line_arrangements(line[:max(pos - 1, 0)], left_clues)
            if left_arrangements:
                arr += left_arrangements * line_arrangements(line[pos + clue + 1:], right_clues)
    return arr

The second solution uses dynamic programming.  
It processes part 2 with the full input in about 100 milliseconds.

In [ ]:
@cache
def line_arrangements(line, clues):
    if len(clues) == 0:
        if "#" in line:
            return 0
        return 1
    if sum(clues) + len(clues) - 1 > len(line):
        return 0

    start = 0
    clue = clues[0]
    while (
        "." in line[start:(stop := start + clue)]
        or (stop < len(line) and line[stop] == "#")
    ):
        if line[start] == "#" or stop >= len(line):
            return 0
        start += 1
    arr = line_arrangements(line[stop + 1:], clues[1:])
    if line[start] == "?":
        arr += line_arrangements(line[start + 1:], clues)
    return arr

In [ ]:
def arrangements(data, repeat=1):
    arr = 0
    for line in data.splitlines():
        springs, clues_str = line.split()
        clues = tuple(int(v) for v in clues_str.split(","))
        arr += line_arrangements("?".join([springs] * repeat), clues * repeat)
    return arr

In [ ]:
check(arrangements, tests)
arrangements(data)

# Part 2

In [ ]:
%%time
check(arrangements, tests, 2, repeat=5)
arrangements(data, repeat=5)